In [1]:
import numpy as np
import requests
import unicodecsv as csv
import os
import pandas as pd
from operator import itemgetter
import random
from IPython.display import Image
from IPython.core.display import HTML 

from IPython.display import display
pd.options.display.max_columns = None

In [2]:
r = requests.get("https://collectionapi.metmuseum.org/public/collection/v1/objects")
r.encoding = r.apparent_encoding
r_json = r.json()
total = r_json['total']
print("There are {} valid objects in this dataset".format(total))
objectIDs = r_json['objectIDs']

There are 492450 valid objects in this dataset


In [3]:
# get the column headers from the first object ID
prefix = "https://collectionapi.metmuseum.org/public/collection/v1/objects/"
url = prefix + str(objectIDs[0])
r = requests.get(url)
col_headers = r.json().keys()

print("The column headers are the following: {}".format(col_headers))

The column headers are the following: [u'locus', u'artistEndDate', u'geographyType', u'objectID', u'locale', u'constituents', u'period', u'dynasty', u'county', u'culture', u'medium', u'portfolio', u'city', u'dimensions', u'title', u'artistDisplayName', u'objectDate', u'primaryImage', u'artistSuffix', u'excavation', u'state', u'department', u'artistNationality', u'classification', u'isHighlight', u'objectName', u'tags', u'objectURL', u'objectBeginDate', u'reign', u'linkResource', u'artistPrefix', u'isPublicDomain', u'artistBeginDate', u'objectEndDate', u'accessionNumber', u'artistDisplayBio', u'rightsAndReproduction', u'artistRole', u'country', u'region', u'creditLine', u'repository', u'additionalImages', u'subregion', u'artistAlphaSort', u'primaryImageSmall', u'river', u'metadataDate']


In [4]:
ID = '52532'
url = prefix + str(ID)
r = requests.get(url)
#r.json().values()[(col_headers.index("isPublicDomain") == True) & (col_headers.index("primaryImage") != '') & (col_headers.index("department") != 'European Paintings')]
#list(itemgetter('isPublicDomain','department')(r.json())) == [True, 'Drawings and Prints']
#list(itemgetter('isPublicDomain','department','primaryImage')(r.json()))
r.json()

{u'accessionNumber': u'50.61.8',
 u'additionalImages': [u'https://images.metmuseum.org/CRDImages/as/original/50_61_8_Tp.jpg'],
 u'artistAlphaSort': u'',
 u'artistBeginDate': u'',
 u'artistDisplayBio': u'',
 u'artistDisplayName': u'',
 u'artistEndDate': u'',
 u'artistNationality': u'',
 u'artistPrefix': u'',
 u'artistRole': u'',
 u'artistSuffix': u'',
 u'city': u'',
 u'classification': u'Ceramics',
 u'constituents': None,
 u'country': u'',
 u'county': u'',
 u'creditLine': u'Harris Brisbane Dick Fund, 1950',
 u'culture': u'China',
 u'department': u'Asian Art',
 u'dimensions': u'H. 7 5/16 in. (18.6 cm); Diam. (at top)  5 7/8 in. (14.9 cm)',
 u'dynasty': u'',
 u'excavation': u'',
 u'geographyType': u'',
 u'isHighlight': False,
 u'isPublicDomain': True,
 u'linkResource': u'',
 u'locale': u'',
 u'locus': u'',
 u'medium': u'Pottery',
 u'metadataDate': u'2019-03-10T18:56:26.507Z',
 u'objectBeginDate': -10000,
 u'objectDate': u'',
 u'objectEndDate': -2100,
 u'objectID': 52532,
 u'objectName': u

In [5]:
#Found item ids from the available csv in GitHub
#isPublicDomain = True
#Specifically looked Departments = Arts of Africa, Oceania, and the Americas,
#Egyptian Art, Islamic Art, Medieval Art
#Medium = pottery or ceramic
objectIDs = open("../Image Collection/data/ObjectIds.txt", "r")
objectIDs = objectIDs.read().splitlines()

In [193]:
'''conditions_list = []
for i in objectIDs:
    url = prefix + str(i)
    r = requests.get(url)
    conditions_list.append(i)
    print(i)'''

'conditions_list = []\nfor i in objectIDs:\n    url = prefix + str(i)\n    r = requests.get(url)\n    conditions_list.append(i)\n    print(i)'

In [7]:
# iterate through the object ids and write the the file
#May take 30 minutes

# create the csv file and write the first row
filename = '../Image Collection/Data/metdata.csv'
# remove if it already exists
try:
    os.remove(filename)
except OSError:
    pass

with open(filename, 'w') as csv_file:
    csv_writer = csv.writer(csv_file)
    csv_writer.writerow(col_headers)
    
    for index, ID in enumerate(objectIDs):
        url = prefix + str(ID)
        r = requests.get(url)
        image_flag = r.json().values()[col_headers.index("primaryImage")]
        if (image_flag != ''):
            values = r.json().values()
            csv_writer.writerow(values)

In [40]:
met_df = pd.read_csv("data/metdata.csv")
met_df['primaryImage'] = met_df['primaryImage'].str.strip()

In [41]:
met_df.head()
met_df.shape

(3018, 49)

In [42]:
#Want to determine the country a piece of pottery was made
#Look at the unique countries in the dataset
met_df.country.value_counts().reset_index()

,index,country
0,Egypt,1055
1,Peru,824
2,Mexico,465
3,Byzantine Egypt,380
4,Ecuador,79
5,United States,55
6,Costa Rica,35
7,Guatemala,15
8,Colombia or Ecuador,14
9,Iran,11


In [49]:
#Can either simplify or remove images from dataset
url= met_df.primaryImage.sample(n=1).values[0]
title = met_df.title[met_df.primaryImage == url]
country = met_df.country[met_df.primaryImage == url]
print(title + ", " + country)
Image(url= url, width=350, height=350)

2967    Single Spout Bottle with Strap Handles, Peru
dtype: object


In [44]:
#Decide to leave countries with less than 8 images in the test set
#Interested in which country the algo will drop them in

In [47]:
# folder to store images
IMAGE_FOLDER = "../image Collection/data/images"
# make the image folder if it doesn't exit

if not os.path.exists(IMAGE_FOLDER):
    os.makedirs(IMAGE_FOLDER)

try:
    os.remove(IMAGE_FOLDER)
except OSError:
    pass

for country, ID, imgurl in zip(met_df['country'][:], met_df['objectID'][:], met_df['primaryImage'][:]):
    image_filename = os.path.join(IMAGE_FOLDER, "{}.jpg".format(str(country) + "_" + str(ID)))
    
    img_data = requests.get(imgurl).content
    with open(image_filename, 'wb') as handler:
        handler.write(img_data)